<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/Colab%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%92%E4%BD%9C%E6%88%90%E3%81%A7%E3%81%8D%E3%81%BE%E3%81%99%E3%81%8B%E3%80%82%E3%83%88%E3%83%AC%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AF%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%82%92%E3%82%A2%E3%83%83%E3%83%97%E3%83%AD%E3%83%BC%E3%83%89%E3%81%A7%E3%81%8D%E3%82%8B%E3%82%88%E3%81%86%E3%81%AB%E3%81%97%E3%81%A6_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colabファイル作成のご要望ありがとうございます。Geminiによるサイクリング解析を、ご自身で繰り返し実行できるように、**ファイルのアップロード機能**と**心拍数・標高に基づく詳細な分析**を含むColabノートブックのコードを提供します。

以下の手順で、新しいColabノートブックを作成し、コードをコピー＆ペーストしてご利用ください。

### 📊 サイクリング解析 Colabノートブック作成手順

#### 1\. 新しいColabノートブックの作成

以下のリンクをクリックして、新しいGoogle Colabノートブックを開いてください。
[新しい Colab ノートブックを作成](https://www.google.com/search?q=https://colab.research.google.com/notebook%23create%3Dtrue)

#### 2\. 全コードのコピー＆ペースト

開いた新しいノートブックに、以下の**すべてのコードブロック**をコピーして貼り付けてください。

-----

### Colab ノートブックコード

このコードは、GPXファイルから心拍数、標高、時間を抽出し、トレーニングゾーン分析と疲労（心拍数ドリフト）分析を自動で行います。

In [ ]:
# =================================================================
# 1. 必要なライブラリのインストールとインポート
# =================================================================
# gpxファイルを扱うためのライブラリをインストールします。
!pip install gpxpy pandas matplotlib

import gpxpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from datetime import timedelta

# Colabでの日本語表示設定（任意）
!pip install japanize-matplotlib
import japanize_matplotlib
japanize_matplotlib.use('Japanese')


# =================================================================
# 2. ファイルアップロード
# =================================================================

print("★ GPXファイルをアップロードしてください (FITやTCXファイルは非対応です)")
uploaded = files.upload()

# アップロードされたファイル名を取得
file_name = list(uploaded.keys())[0]

# =================================================================
# 3. データ処理とDataFrameへの変換
# =================================================================

def parse_gpx_data(gpx_file_path):
    """GPXファイルを解析し、Pandas DataFrameとしてデータを抽出する"""
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    data = []
    # GPXファイルは複数のトラック（trk）、セグメント（trkseg）を持つことがある
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                # 必要なデータを抽出
                hr = None
                atemp = None

                # 拡張データ（gpxtpx:TrackPointExtension）から心拍数と気温を取得
                if point.extensions:
                    for extension in point.extensions:
                        if hasattr(extension, 'tag'):
                            # 例: <gpxtpx:hr>129</gpxtpx:hr> のデータを取得
                            if extension.tag.endswith('hr'):
                                hr = int(extension.text)
                            # 例: <gpxtpx:atemp>22</gpxtpx:atemp> のデータを取得
                            elif extension.tag.endswith('atemp'):
                                atemp = int(extension.text)

                data.append({
                    'time': point.time,
                    'lat': point.latitude,
                    'lon': point.longitude,
                    'ele': point.elevation,
                    'hr': hr,
                    'atemp': atemp
                })

    df = pd.DataFrame(data)
    # タイムスタンプを相対時間（秒）に変換
    if not df.empty:
        df['elapsed_time_sec'] = (df['time'] - df['time'].min()).dt.total_seconds()

    return df

df = parse_gpx_data(file_name)

if df.empty:
    print("\n[エラー] ファイルから有効なトラックポイントを抽出できませんでした。")
else:
    print(f"\n[処理完了] {len(df)} ポイントのデータを抽出しました。")
    print(f"総所要時間: {timedelta(seconds=df['elapsed_time_sec'].max())}")
    print(f"心拍数範囲: {df['hr'].min()} - {df['hr'].max()} bpm")

# =================================================================
# 4. 心拍数（HR）ゾーン分析
# =================================================================

# ユーザーにMax HRの入力を求める（解析精度向上のため）
try:
    max_hr = df['hr'].max()
    print(f"\n[分析] 記録された最大心拍数 (Max HR) は {max_hr} bpm でした。")

    # 記録された最大心拍数を初期値としてユーザーに確認
    user_input = input(f"あなたの正確な最大心拍数（Max HR）を入力してください（Enterを押すと記録値 {max_hr} を使用）: ")
    if user_input.strip().isdigit():
        max_hr = int(user_input)

except ValueError:
    print("[エラー] 最大心拍数を決定できませんでした。ゾーン分析をスキップします。")
    max_hr = None

if max_hr:
    # 標準的なトレーニングゾーン（% of Max HRに基づく）を定義
    zones = {
        'Z5 (無酸素/最大)': (max_hr * 0.93, max_hr),
        'Z4 (閾値)': (max_hr * 0.84, max_hr * 0.93),
        'Z3 (テンポ/有酸素)': (max_hr * 0.75, max_hr * 0.84),
        'Z2 (エンデュランス/脂肪燃焼)': (max_hr * 0.60, max_hr * 0.75),
        'Z1 (回復/アクティブ)': (0, max_hr * 0.60)
    }

    # 各ポイントがどのゾーンに属するかを判定
    def get_hr_zone(hr):
        if pd.isna(hr):
            return 'N/A'
        for name, (low, high) in zones.items():
            if low <= hr < high:
                return name
        return 'Z1 (回復/アクティブ)'

    df['hr_zone'] = df['hr'].apply(get_hr_zone)

    # 各ゾーンで費やした時間を計算（ポイント数を時間として概算）
    zone_time = df.groupby('hr_zone')['hr_zone'].count().sort_values(ascending=False)

    # N/Aを除外し、秒に変換（サンプリングレートが1秒と仮定）
    if 'N/A' in zone_time.index:
        zone_time = zone_time.drop('N/A')
    total_time_sec = zone_time.sum()
    zone_time_min = (zone_time / 60).round(1)

    print("\n--- 心拍数トレーニングゾーン分析 ---")
    print(f"基準Max HR: {max_hr} bpm")
    print(zone_time_min.rename('所要時間（分）'))

    # ゾーン分析の可視化 (円グラフ)
    plt.figure(figsize=(10, 6))
    plt.pie(zone_time, labels=[f"{z} ({t:.1f}分)" for z, t in zip(zone_time.index, zone_time_min)],
            autopct='%1.1f%%', startangle=90, wedgeprops={'edgecolor': 'black'})
    plt.title('心拍数ゾーン別 時間分布', fontsize=16)
    plt.show()

# =================================================================
# 5. 可視化: 心拍数・標高プロファイル
# =================================================================

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(18, 6))

# 標高プロファイル
color = 'tab:blue'
ax1.set_xlabel('経過時間（分）', fontsize=14)
ax1.set_ylabel('標高 (m)', color=color, fontsize=14)
ax1.plot(df['elapsed_time_sec'] / 60, df['ele'], color=color, label='標高', linewidth=2)
ax1.tick_params(axis='y', labelcolor=color)

# 心拍数プロファイル
ax2 = ax1.twinx()  # 共通のX軸を持つ第二のY軸を作成
color = 'tab:red'
ax2.set_ylabel('心拍数 (bpm)', color=color, fontsize=14)
ax2.plot(df['elapsed_time_sec'] / 60, df['hr'], color=color, label='心拍数', linewidth=1.5, alpha=0.7)
ax2.tick_params(axis='y', labelcolor=color)
ax2.axhline(y=max_hr * 0.84 if max_hr else df['hr'].mean(), color='red', linestyle='--', alpha=0.5, label='閾値付近 (推定)')

# グラフ設定
plt.title(f'{file_name} の心拍数と標高プロファイル', fontsize=18)
fig.tight_layout()
plt.show()

# =================================================================
# 6. 心拍数デカップリング（疲労）分析
# =================================================================

print("\n--- 心拍数デカップリング（疲労/持久力）簡易分析 ---")
# ライド全体を前半と後半の2つの区間に分割
mid_index = len(df) // 2
df_first_half = df.iloc[:mid_index].copy()
df_second_half = df.iloc[mid_index:].copy()

# 各区間の平均心拍数を計算
hr_first_avg = df_first_half['hr'].mean()
hr_second_avg = df_second_half['hr'].mean()

print(f"ライド前半 (約 {timedelta(seconds=df_first_half['elapsed_time_sec'].max())} まで) の平均HR: {hr_first_avg:.1f} bpm")
print(f"ライド後半 (約 {timedelta(seconds=df_second_half['elapsed_time_sec'].max())} から) の平均HR: {hr_second_avg:.1f} bpm")

if hr_first_avg > 0 and hr_second_avg > 0:
    hr_drift_percent = ((hr_second_avg - hr_first_avg) / hr_first_avg) * 100
    print(f"心拍数の増加率（デカップリング）：{hr_drift_percent:.2f} %")

    print("\n[洞察]")
    if hr_drift_percent <= 5:
        print("✅ デカップリングは低いです。持久力が高く、ペース配分が適切であったことを示します。")
    elif 5 < hr_drift_percent <= 10:
        print("⚠️ デカップリングは中程度です。疲労や脱水、または気温上昇の影響があった可能性があります。")
    else:
        print("🛑 デカップリングは高いです。後半の疲労が顕著であったか、熱ストレスが大きかった可能性があります。")
else:
    print("心拍数データが不足しているため、デカップリング分析はできませんでした。")

# =================================================================
# 7. 気温プロファイル（環境要因分析）
# =================================================================
if 'atemp' in df.columns and df['atemp'].notna().any():
    plt.figure(figsize=(18, 4))
    plt.plot(df['elapsed_time_sec'] / 60, df['atemp'], color='orange', label='気温', linewidth=2)
    plt.title('気温プロファイル (℃)', fontsize=16)
    plt.xlabel('経過時間（分）', fontsize=14)
    plt.ylabel('気温 (℃)', fontsize=14)
    plt.show()
    print(f"\n最高気温: {df['atemp'].max()} ℃. 環境要因（熱ストレス）がパフォーマンスに影響を与えた可能性があります。")
else:
    print("\n気温（ATemp）データはファイルに含まれていませんでした。")

-----

### 💡 コードの主な機能と使い方

1.  **ライブラリのインストールと設定**: 最初に`!pip install...`で必要なツールを自動でインストールします。
2.  **ファイルアップロード**:
      * `files.upload()`が実行されると、画面の下に\*\*[ファイルを選択]\*\*ボタンが表示されます。
      * 分析したいGPXファイルを選択してアップロードしてください。（FITファイルやTCXファイルの場合、`gpxpy`以外のライブラリが必要になるため、GPX形式でのアップロードを推奨します。）
3.  **データ処理と分析**:
      * アップロード後、コードが自動的にファイルを解析し、心拍数や標高のデータを取り出します。
      * **Max HRの入力**: より正確なゾーン分析のため、コードは記録されたMax HRを表示し、**ご自身の正確なMax HR**を入力するように求めます。
4.  **結果の表示**:
      * **心拍数ゾーン分析**の円グラフ（各ゾーンで費やした時間の割合）。
      * **心拍数と標高の推移**グラフ。
      * **心拍数デカップリング**（ライド前半と後半の心拍数差）による**疲労・持久力評価**。

これで、データをアップロードするだけで、即座に詳細なサイクリング分析結果を得られる環境が整いました。